In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
from sklearn.preprocessing import MinMaxScaler
import datetime

In [2]:
combined_data = pd.read_csv('../data/intermediates/pre_safegraph.csv')
combined_data.set_index('FIPS', inplace=True)
combined_data.head()

,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Jan Temp AVG / F,Feb Temp AVG / F,Mar Temp AVG / F,Apr Temp AVG / F,...,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,Area in square miles - Land area,NursingCt,HospCt,Hispanic Population,Black Population,Deaths,Outbreak_date
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,91.8,2.0,11.3,27.7,3.6,119.0,47.0,58.6,56.1,65.0,...,NaN,NaN,55601,594.44,6.0,85.0,1649.0,10915.0,4,2020-04-27
1003,114.6,3.0,9.7,31.3,3.6,115.5,52.0,62.0,59.7,66.0,...,11209.0,6503.0,218022,1589.78,9.0,152.0,10131.0,19492.0,6,2020-04-20
1011,17.5,6.0,24.8,13.3,4.7,58.7,48.0,58.2,56.6,64.3,...,NaN,NaN,10138,622.80,1.0,61.0,807.0,7168.0,8,2020-05-25
1013,27.0,6.0,15.4,16.1,4.8,74.9,48.4,59.3,57.0,64.4,...,NaN,NaN,19680,776.83,5.0,94.0,297.0,8839.0,16,2020-05-08
1015,195.7,3.0,15.9,18.0,4.7,91.0,44.2,52.6,53.0,63.0,...,4972.0,6890.0,114277,605.87,9.0,152.0,4469.0,24153.0,3,2020-04-20


In [3]:
# path = '/Users/beatrizmedeiros/Johns Hopkins/Fardin Ganjkhanloo - Number of trips/'

# for i in combined_data.index:
#     print(i)
#     outbreak_date = combined_data.loc[i, 'Outbreak_date']
#     trips = pd.read_csv(path + outbreak_date.strftime('%Y_%m_%d') + '.csv', index_col=0)
#     intra_mob = trips.loc[i, str(i)]
#     inter_mob = np.sum(trips.loc[i]) + np.sum(trips[str(i)])
#     print(intra_mob, inter_mob)
# #     print(trips.loc[i])
# #     print(trips[str(i)])
# #     print(trips.head())
#     break

In [4]:
path = '/Users/michaellepori/Documents/COVID/covid-drivers-master/data/intermediates/Number of trips/'
sg_mobility = pd.DataFrame(index=combined_data.index, columns=['2wk Prior Intra-Mobility', '2wk Onset Intra-Mobility', '2wk Prior Inter-Mobility', '2wk Onset Inter-Mobility'])
# print(sg_mobility.head())
for i in combined_data.index: 
    print(i)
    inter_mob_per_day = np.zeros(21)
    intra_mob_per_day = np.zeros(21)
    row = combined_data.loc[i]
    outbreak_date = datetime.datetime.strptime(row['Outbreak_date'], '%Y-%m-%d')
    date1 = 14
    date2 = 7
    for j in range(-date1,date2): 
        mob_day = outbreak_date + datetime.timedelta(days=j) #.AddDays(j)
        mob_day = mob_day.date()
        try: 
            trips = pd.read_csv(path + mob_day.strftime('%Y_%m_%d') + '.csv', index_col=0)
        except: 
            print('missing mobility data for', mob_day.strftime('%Y_%m_%d'))
            if j > -date1: 
                inter_mob_per_day[j+date1] = inter_mob_per_day[j+date1-1]
                intra_mob_per_day[j+date1] = intra_mob_per_day[j+date1-1]
            else: 
                try: 
                    trips = pd.read_csv(path + outbreak_date.strftime('%Y_%m_%d') + '.csv', index_col=0)
                except: 
                    print('missing mobility data for outbreak date:', outbreak_date.strftime('%Y_%m_%d'))
                    continue
            continue
        intra_mob = trips.loc[i, str(i)]
        inter_mob = np.sum(trips.loc[i]) + np.sum(trips[str(i)]) - intra_mob # trips between this county and all OTHER counties 
        inter_mob_per_day[j+date1] = inter_mob
        intra_mob_per_day[j+date1] = intra_mob
    
    inter_prior_mob_arr = np.array(inter_mob_per_day[date1:date1+14])
    inter_onset_mob_arr = np.array(inter_mob_per_day[date1-7:-7])
    
    inter_prior_mob = np.mean(inter_prior_mob_arr)
    inter_onset_mob = np.mean(inter_onset_mob_arr)
    
    intra_prior_mob_arr = np.array(intra_mob_per_day[date1:date1+14])
    intra_onset_mob_arr = np.array(intra_mob_per_day[date1-7:-7])
    
    intra_prior_mob = np.mean(intra_prior_mob_arr)
    intra_onset_mob = np.mean(intra_onset_mob_arr)
    
    sg_mobility.loc[i,'2wk Prior Inter-Mobility'] = inter_prior_mob
    sg_mobility.loc[i, '2wk Onset Inter-Mobility'] = inter_onset_mob
    
    sg_mobility.loc[i,'2wk Prior Intra-Mobility'] = intra_prior_mob
    sg_mobility.loc[i, '2wk Onset Intra-Mobility'] = intra_onset_mob
    if i == 1003:
        break
#     if i < combined_data.index[5]:
#         print(mob_per_day)
#         print(prior_mob_arr, prior_mob)
#         print(onset_mob_arr, onset_mob)
#         print(pos_mob_arr, pos_mob)
sg_mobility.head()

1001
1003


,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility
FIPS,,,,
1001,7461.57,7314.29,13927.1,13194.9
1003,35004,32185.7,47460.3,42444.7
1011,NaN,NaN,NaN,NaN
1013,NaN,NaN,NaN,NaN
1015,NaN,NaN,NaN,NaN


In [ ]:
sg_mobility.to_csv('../data/processed/safegraph_mobility_avgs_ml_check.csv')